In [1]:
%matplotlib inline
%run ../../path_import.py
import numpy as np
import matplotlib.pyplot as plt
from plots import *
from utils import *
from training import *
from model_mu_reparam import *
import time
import probtorch
print('probtorch:', probtorch.__version__, 
      'torch:', torch.__version__, 
      'cuda:', torch.cuda.is_available())

probtorch: 0.0+5a2c637 torch: 1.1.0 cuda: True


In [2]:
Data = torch.from_numpy(np.load('../rings_fixed_radius/obs.npy')).float()
## Data Parameters
N = 300
K = 3
D = 2
FIXED_RADIUS = 1.5
## Model Parameters
MCMC_SIZE = 10
SAMPLE_SIZE = 10
NUM_HIDDEN_GLOBAL = 8
NUM_HIDDEN_LOCAL = 64
STAT_SIZE = 8
NUM_LATENTS =  D
## Training Parameters
BATCH_SIZE = 20
NUM_EPOCHS = 500
LEARNING_RATE =  1e-4
CUDA = torch.cuda.is_available()
PATH = 'ag-pr-%dsteps-%dsamples' % (MCMC_SIZE, SAMPLE_SIZE)
DEVICE = torch.device('cuda:1')

In [3]:
enc_mu = Enc_mu(K, D, num_hidden=NUM_HIDDEN_GLOBAL, num_stats=STAT_SIZE, CUDA=CUDA, device=DEVICE)
enc_z = Gibbs_z(K, CUDA, DEVICE)
obs_rad = torch.ones(1) * FIXED_RADIUS
noise_sigma = torch.ones(1) * 0.05
if CUDA:
    enc_mu.cuda().to(DEVICE)
    obs_rad = obs_rad.cuda().to(DEVICE)
    noise_sigma =noise_sigma.cuda().to(DEVICE)
optimizer =  torch.optim.Adam(list(enc_mu.parameters()),lr=LEARNING_RATE, betas=(0.9, 0.99))

In [ ]:
RESAMPLE=True
DETACH=True
train_mu(Eubo_cfz_pr_z, enc_mu, enc_z, optimizer, Data, obs_rad, noise_sigma, K, NUM_EPOCHS, MCMC_SIZE, SAMPLE_SIZE, BATCH_SIZE, PATH, CUDA, DEVICE, RESAMPLE=RESAMPLE, DETACH=DETACH)

epoch=0, SymKL=1166373.516, EUBO=-258110.706, ELBO=-1510030.335, ESS=1.010 (23s)
epoch=1, SymKL=1005253.481, EUBO=-244840.758, ELBO=-1324715.569, ESS=1.012 (26s)
epoch=2, SymKL=872714.209, EUBO=-236196.500, ELBO=-1175894.179, ESS=1.012 (25s)
epoch=3, SymKL=767468.615, EUBO=-225218.946, ELBO=-1054044.773, ESS=1.011 (24s)
epoch=4, SymKL=677790.402, EUBO=-212560.517, ELBO=-947198.722, ESS=1.012 (24s)
epoch=5, SymKL=598250.773, EUBO=-201053.980, ELBO=-852408.623, ESS=1.014 (25s)
epoch=6, SymKL=529697.426, EUBO=-188992.320, ELBO=-768429.103, ESS=1.012 (25s)
epoch=7, SymKL=465776.964, EUBO=-179091.306, ELBO=-691880.429, ESS=1.011 (26s)
epoch=8, SymKL=409226.877, EUBO=-165853.636, ELBO=-620196.736, ESS=1.010 (25s)
epoch=9, SymKL=355354.664, EUBO=-155162.747, ELBO=-553609.895, ESS=1.009 (25s)
epoch=10, SymKL=306952.972, EUBO=-145097.328, ELBO=-493016.881, ESS=1.008 (25s)
epoch=11, SymKL=264511.499, EUBO=-134872.698, ELBO=-438981.671, ESS=1.007 (26s)
epoch=12, SymKL=227660.204, EUBO=-127030.353

In [ ]:
torch.save(enc_mu.state_dict(), "../weights/enc-mu-%s" % PATH)
# torch.save(enc_z.state_dict(), "../weights/enc-z-%s" % PATH)

In [ ]:
BATCH_SIZE_TEST = 50
obs, q_mu, q_z, symkls_test, eubos_test, elbos_test = test(Eubo_cfz_init_eta, enc_mu, enc_z, Data, obs_rad, noise_sigma, K, 10, SAMPLE_SIZE, BATCH_SIZE_TEST, CUDA, DEVICE, RESAMPLE, DETACH=DETACH)
%time plot_samples(obs, q_mu, q_z, K, PATH)

In [ ]:
incremental_gap = symkls_test.cpu().data.numpy()[1:]
M = incremental_gap.shape[0]
overall_gap = np.zeros(M)
for m in range(M):
    overall_gap[m] = incremental_gap[:m+1].sum()

In [ ]:
fig = plt.figure(figsize=(12,6))
ax = fig.add_subplot(111)
plt.yscale("log")
ax.plot(incremental_gap, label="incremental gap")
ax.plot(overall_gap, label='overall gap')
ax.legend(fontsize=14)
ax.set_xlabel('Steps')